# 영업 성공 여부 분류 경진대회

# 할 것

결측치 이상치 처리
파생변수 생성

## 1. 데이터 확인

### 필수 라이브러리

In [ ]:
# !pip uninstall googletrans -y
# !pip install googletrans==3.1.0a0 

In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

from googletrans import Translator
translator = Translator()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def seed_everything(seed):
    np.random.seed(seed) #numpy를 사용할 경우 고정
seed_everything(42)

### 데이터 셋 읽어오기

In [2]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [3]:
df_train[:10]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_country.1,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,NaN,purchasing,62,Quotation or purchase consultation,multi-split,NaN,NaN,/Quezon City/Philippines,entry level,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.00,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,NaN,media and communication,96,Quotation or purchase consultation,multi-split,NaN,NaN,/PH-00/Philippines,ceo/founder,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.00,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,NaN,engineering,56,Product Information,single-split,NaN,NaN,/Kolkata /India,partner,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.00,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,NaN,entrepreneurship,44,Quotation or purchase consultation,vrf,NaN,NaN,/Bhubaneswar/India,ceo/founder,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.00,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,NaN,consulting,97,Quotation or purchase consultation,multi-split,NaN,NaN,/Hyderabad/India,partner,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True
5,1.00,/Abuja/Nigeria,AS,0.040816,16328,End-Customer,SMB,NaN,NaN,NaN,NaN,program and project management,1114,Quotation or purchase consultation,chiller,NaN,NaN,/Abuja/Nigeria,manager,LGEAF,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,5,True
6,0.75,"/Jeddah, KSA/Saudi Arabia",AS,0.040816,20664,End-Customer,SMB,NaN,NaN,NaN,NaN,engineering,420,Quotation or purchase consultation,single-split,NaN,NaN,"/Jeddah, KSA/Saudi Arabia",manager,LGESJ,NaN,1,0,0.003079,0.026846,corporate / office,Engineering,6,True
7,1.00,/Guwahati/India,AS,0.088889,17983,End-Customer,SMB,NaN,NaN,NaN,NaN,sales,205,Quotation or purchase consultation,vrf,NaN,NaN,/Guwahati/India,partner,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Manufacturing,7,True
8,0.75,/Cebu city/Philippines,AS,0.066667,30867,Specifier/ Influencer,Enterprise,3.0,NaN,NaN,NaN,other,103,Quotation or purchase consultation,multi-split,NaN,NaN,/Cebu city/Philippines,vice president,LGEPH,less than 3 months,0,0,0.003079,0.026846,corporate / office,Construction,8,True
9,0.75,"/hauz khas,delhi/India",AS,0.088889,6084,End-Customer,SMB,NaN,NaN,NaN,NaN,other,252,Quotation or purchase consultation,vrf,NaN,NaN,"/hauz khas,delhi/India",manager,LGEIL,3 months ~ 6 months,1,0,0.003079,0.026846,corporate / office,NaN,9,True


In [4]:
# 결측치가 있는 변수 및 결측률 파악 

mis_val = df_train.isnull().sum()
mis_val_bool = mis_val >= 1
mis_val_df = pd.concat([mis_val, mis_val_bool], axis = 1)
mis_val_df = mis_val_df.rename(columns = {0 : 'mis_val', 1 : 'mis_val_bool'})

mis_val_data = mis_val_df.loc[mis_val_df['mis_val_bool'] == True, :]
mis_val_data['ratio'] = mis_val_data['mis_val'] / 59299
mis_val_data_sorted = mis_val_data.sort_values(by='ratio', ascending=True)
mis_val_data_sorted

/tmp/ipykernel_231/929980573.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mis_val_data['ratio'] = mis_val_data['mis_val'] / 59299


,mis_val,mis_val_bool,ratio
inquiry_type,941,True,0.015869
customer_country.1,982,True,0.016560
customer_country,982,True,0.016560
customer_job,18733,True,0.315908
product_category,19374,True,0.326717
expected_timeline,30863,True,0.520464
business_area,40882,True,0.689421
ver_win_rate_x,40882,True,0.689421
customer_type,43961,True,0.741345
ver_win_ratio_per_bu,43995,True,0.741918


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            15338 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             40566 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             58358 non-null  object 
 14  product_category      

### 전처리

## 추가된 전처리

In [6]:
categorical_features = ['business_unit', "customer_country",
                        'customer_idx', 'customer_type',
                        'enterprise', 'customer_job', 'inquiry_type',
                        'product_category', 'product_subcategory', 'product_modelname',
                        'customer_position', 'response_corporate', 'expected_timeline',
                        'business_area', 'business_subarea', 'lead_owner']

numerical_features = ['bant_submit', 'com_reg_ver_win_rate',
                      'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
                      'idit_strategic_ver', 'lead_desc_length',
                      'ver_cus', 'ver_pro', 'ver_win_rate_x', 'ver_win_ratio_per_bu']

replace_words = {
    'others': 'other',
    'etc': 'other',
    'none': 'other',
    'not specified' : 'other',
    'other store' : 'other'
}

In [7]:
# customer_country , customer_country.1의 값이 일치함.
pre_train = df_train.drop(labels = ['customer_country.1'],axis=1)
pre_test = df_test.drop(labels = ['customer_country.1','id'],axis=1)

dtypes = pre_train.dtypes

# 각 열의 고유값 개수 확인
unique_value_counts = pre_train.nunique()

# 수치형 변수가 아닌 컬럼을 저장할 리스트 초기화
object_columns = []

# 각 열에 대해 데이터 타입이 'object'이거나 고유값 개수가 일정 이하인 경우를 확인하여 non_numeric_columns 리스트에 추가
for col in pre_train.columns:
    if dtypes[col] == 'object':
        object_columns.append(col)

# 수치형 변수가 아닌 컬럼 출력
print("범주형 컬럼:", object_columns,"\n")
print("수치형 컬럼 :", list(set(pre_train.columns)-set(object_columns)))

범주형 컬럼: ['customer_country', 'business_unit', 'customer_type', 'enterprise', 'customer_job', 'inquiry_type', 'product_category', 'product_subcategory', 'product_modelname', 'customer_position', 'response_corporate', 'expected_timeline', 'business_area', 'business_subarea'] 

수치형 컬럼 : ['com_reg_ver_win_rate', 'lead_owner', 'idit_strategic_ver', 'ver_win_rate_x', 'ver_win_ratio_per_bu', 'it_strategic_ver', 'lead_desc_length', 'ver_cus', 'customer_idx', 'bant_submit', 'historical_existing_cnt', 'id_strategic_ver', 'ver_pro', 'is_converted']



### 증강

여러 카테고리를 가진 값을 , 기준으로 분해 -> 증강

In [8]:
# 'product_category' 열의 각 값을 쉼표로 분할합니다.
pre_train['product_category'] = pre_train['product_category'].str.split(',')

# explode 함수를 사용하여 각 분할된 값을 별도의 행으로 만듭니다.
pre_train = pre_train.explode('product_category')

In [9]:
pre_train.loc[22696]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
22696,1.0,/Lagos/Nigeria,Solution,NaN,25309,NaN,Enterprise,NaN,NaN,NaN,NaN,real estate,18,Request for quotation or purchase,monior/monitor tv,NaN,NaN,other,LGEKR,9 months ~ 1 year,0,0,0.000543,NaN,special purpose,NaN,441,False
22696,1.0,/Lagos/Nigeria,Solution,NaN,25309,NaN,Enterprise,NaN,NaN,NaN,NaN,real estate,18,Request for quotation or purchase,tv,NaN,NaN,other,LGEKR,9 months ~ 1 year,0,0,0.000543,NaN,special purpose,NaN,441,False
22696,1.0,/Lagos/Nigeria,Solution,NaN,25309,NaN,Enterprise,NaN,NaN,NaN,NaN,real estate,18,Request for quotation or purchase,commercial tv,NaN,NaN,other,LGEKR,9 months ~ 1 year,0,0,0.000543,NaN,special purpose,NaN,441,False
22696,1.0,/Lagos/Nigeria,Solution,NaN,25309,NaN,Enterprise,NaN,NaN,NaN,NaN,real estate,18,Request for quotation or purchase,pc,NaN,NaN,other,LGEKR,9 months ~ 1 year,0,0,0.000543,NaN,special purpose,NaN,441,False
22696,1.0,/Lagos/Nigeria,Solution,NaN,25309,NaN,Enterprise,NaN,NaN,NaN,NaN,real estate,18,Request for quotation or purchase,refrigerator,NaN,NaN,other,LGEKR,9 months ~ 1 year,0,0,0.000543,NaN,special purpose,NaN,441,False
22696,1.0,/Lagos/Nigeria,Solution,NaN,25309,NaN,Enterprise,NaN,NaN,NaN,NaN,real estate,18,Request for quotation or purchase,solar,NaN,NaN,other,LGEKR,9 months ~ 1 year,0,0,0.000543,NaN,special purpose,NaN,441,False
22696,1.0,/Lagos/Nigeria,Solution,NaN,25309,NaN,Enterprise,NaN,NaN,NaN,NaN,real estate,18,Request for quotation or purchase,rac,NaN,NaN,other,LGEKR,9 months ~ 1 year,0,0,0.000543,NaN,special purpose,NaN,441,False
22696,1.0,/Lagos/Nigeria,Solution,NaN,25309,NaN,Enterprise,NaN,NaN,NaN,NaN,real estate,18,Request for quotation or purchase,washing machine,NaN,NaN,other,LGEKR,9 months ~ 1 year,0,0,0.000543,NaN,special purpose,NaN,441,False
22696,1.0,/Lagos/Nigeria,Solution,NaN,25309,NaN,Enterprise,NaN,NaN,NaN,NaN,real estate,18,Request for quotation or purchase,mobile,NaN,NaN,other,LGEKR,9 months ~ 1 year,0,0,0.000543,NaN,special purpose,NaN,441,False
22696,1.0,/Lagos/Nigeria,Solution,NaN,25309,NaN,Enterprise,NaN,NaN,NaN,NaN,real estate,18,Request for quotation or purchase,ess,NaN,NaN,other,LGEKR,9 months ~ 1 year,0,0,0.000543,NaN,special purpose,NaN,441,False


In [10]:
import string
import re

def remove_punctuation(text):
    if pd.isna(text):
        return np.nan

    text = re.sub('[~_-]', ' ', text)
    text = re.sub('[,&(]', ' ', text)
    text = re.sub('[).]', ' ', text)
    text = text.rstrip()
    text = text.lstrip()
    text = re.sub('\s\s', ' ', text)
    text = re.sub('\s\s', ' ', text)  # 띄어쓰기 연속 제거

    return text

def remove_slash(text):
    if pd.isna(text):
        return np.nan
    return re.sub('/', ' ', text)

remove_slash('dfh/dfdg')

'dfh dfdg'

In [11]:
# 결측치 제거
pre_train['historical_existing_cnt'] = pre_train['historical_existing_cnt'].fillna(pre_train['historical_existing_cnt'].median())
pre_test['historical_existing_cnt'] = pre_test['historical_existing_cnt'].fillna(pre_test['historical_existing_cnt'].median())

pre_train[numerical_features] = pre_train[numerical_features].fillna(0)
pre_test[numerical_features] = pre_test[numerical_features].fillna(0)
pre_train[categorical_features] = pre_train[categorical_features].fillna('other')
pre_test[categorical_features] = pre_test[categorical_features].fillna('other')

In [12]:
pre_train["customer_idx"] = pre_train["customer_idx"].astype(str)
pre_train["lead_owner"] = pre_train["lead_owner"].astype(str)

pre_test["customer_idx"] = pre_test["customer_idx"].astype(str)
pre_test["lead_owner"] = pre_test["lead_owner"].astype(str)

for feature in categorical_features:
  pre_train[feature] = pre_train[feature].str.lower()
  pre_test[feature] = pre_test[feature].str.lower()

  pre_train[feature] = pre_train[feature].apply(remove_punctuation)
  pre_test[feature] = pre_test[feature].apply(remove_punctuation)

  pre_train[feature] = pre_train[feature].replace(replace_words)
  pre_test[feature] = pre_test[feature].replace(replace_words)

In [13]:
# 범주형으로 변경
columns_categorized = ['it_strategic_ver','id_strategic_ver','idit_strategic_ver'] + ['lead_owner','ver_cus','ver_pro','customer_idx']

for n in columns_categorized:
    pre_train[n] = pre_train[n].fillna(0)
    pre_train[n] = pre_train[n].astype('int').astype('object')

## 언어 통일 (00 -> 영어)

customer_job , inquiry_type , product_category , product_subcategory , product_modelname

시험 코드 

### process about inquiry

In [14]:
pre_train['inquiry_type'].unique()

array(['quotation or purchase consultation', 'product information',
       'other', 'usage or technical consultation', 'trainings',
       'services', 'sales inquiry', 'technical support',
       'technical consultation', 'request for partnership', 'sales',
       'technical', 'request a demo', 'request for distributorship',
       'request for quotation or purchase',
       'request for technical consulting', 'select id needs',
       'one quick:flex', 'aio', 'needs', 'purchase',
       'customer suggestions', 'event inquiry', 'oem/odm request',
       'hospital tv', 'i want to know the details about it',
       'educational equipments', 'digital platform', 'tv interactive',
       'teach', 'display textbook and photos', 'high inch 86 / 98 or 110',
       'quotation', 'purchase or quotation', 'display product',
       'first info and pricing',
       'estoy buscando para ecuador este producto lg magnit micro led para un cliente de 138 pulgadas con envió marítimo',
       'hola me pued

In [15]:
# Other, Others, Etc., ETC., Not specified  >> 똑같은 데이터로 처리 : other
# 결측치 처리 >> other
def process_etc(data):
    if data in [
        'other', 'others', 'etc.', 'not specified', 'none',
        'Other Stores', 'Others', 'nan', 'etc', 'no longer available','not applicable','no influence',
        ]:
        return np.nan
    else:
        return data

def process_inquiry(df):
  df['inquiry_type'].replace({
      'purchase' : 'quotation or purchase consultation',
      'quotation' : 'quotation or purchase consultation',
      'purchase or quotation' : 'quotation or purchase consultation',
      'hola me pueden cotizar 19 pantallas interactivas de 100 pulgadas entregadas en guayaquil ecuador' : 'quotation or purchase consultation',
      'vui lòng báo giá giúp mình sản phẩm đo thân nhiệt xin cảm ơn' : 'quotation or purchase consultation',
      'probeam precio' : 'quotation or purchase consultation',
      'solicito apoyo para realizar cotizacion de los dispositivos que ofrecen en la solución\xa0one quick:' : 'quotation or purchase consultation',

      'usage_or_technical_consultation' : 'usage or technical consultation',
      'tôi cần tham khảo giá và giải pháp từ lg' : 'usage or technical consultation',

      'technical support' : 'technical consultation',
      'technical' : 'technical consultation',

      'one quick:flex' : 'product information',
      'aio' : 'product information',
      'i want to know the details about it' : 'product information',
      'hospital tv' : 'product information',
      'educational equipments' : 'product information',
      'tv interactive' : 'product information',
      'digital platform' : 'product information',
      'high inch 86 / 98 or 110' : 'product information',
      'display textbook and photos' : 'product information',
      'display product' : 'product information',
      'estoy buscando para ecuador este producto lg magnit micro led para un cliente de 138 pulgadas con envió marítimo' : 'product information',
      'window facing product' : 'product information',
      'hotel tv products' : 'product information',
      'pantallas interactivas para clinicas' : 'product information',
      'toi muon tim hieu thong tin ky thuat gia ca cua sp de su dung' : 'product information',
      'preciso de um monitor médico para radiografia convencional e tomogrtafia' : 'product information',
      'led signage' : 'product information',
      'for school' : 'product information',
      'video wall' : 'product information',
      'first info and pricing' : 'product information',
      'idb' : 'product information',
      'vrf' : 'product information',
      'standalone' : 'product information',

      'evento sdelestero' : 'event inquiry',

      'select id needs' : 'other',
      'needs' : 'other',
      'teach' : 'other',
      'event inquiry' : 'other',
      'media inquiry' : 'other',
      'intégrateur historique du george v' : 'other'

      }, inplace = True)


# 동일 범주형 데이터에 대한 대소문자 구별 없애기
pre_train['inquiry_type'] = pre_train['inquiry_type'].str.lower()
pre_train['inquiry_type'] = pre_train['inquiry_type'].apply(process_etc)


pre_test['inquiry_type'] = pre_test['inquiry_type'].str.lower()
pre_test['inquiry_type'] = pre_test['inquiry_type'].apply(process_etc)

# 학습 데이터프레임 처리
process_inquiry(pre_train)

# 테스트 데이터프레임 처리
process_inquiry(pre_test)

In [16]:
pre_train['inquiry_type'].unique()

array(['quotation or purchase consultation', 'product information', nan,
       'usage or technical consultation', 'trainings', 'services',
       'sales inquiry', 'technical consultation',
       'request for partnership', 'sales', 'request a demo',
       'request for distributorship', 'request for quotation or purchase',
       'request for technical consulting', 'other',
       'customer suggestions', 'oem/odm request', 'event inquiry'],
      dtype=object)

### process of  customer position

In [17]:
# 공통 함수 정의
def process_position(df):
    df['customer_position'].replace({
        'ceo/fundador': 'ceo founder',
        'ceo/founder' : 'ceo founder',
        'co founder': 'ceo founder',
        'mindenes': 'ceo founder',
        'genel müdür': 'ceo founder',
        'leadership/executive office/owner': 'ceo founder',
        'the big boss': 'ceo founder',
        'entrepreneur' : 'ceo founder',
        'entrepreneurship' :'ceo founder',
        'business development': 'ceo founder',
        'chairman': 'ceo founder',
        'president' : 'ceo founder',

        'decision influencer': 'decision-influencer',
        'decision-influence' : 'decision-influencer',
        'decision-maker': 'decision-influencer',
        'decision maker': 'decision-influencer',



        'vp': 'vice president',
        'vicepresident':'vice president',
        'visepresident' : 'vice president',
        'gerente': 'vice president',

        'architecture/consult' : 'consultant',
        'architect/consultant': 'consultant',
        'consulting': 'consultant',

        'asst prof' : 'assistant professor',
        'prof' : 'professor',
        'education' : 'professor',
        'educator' : 'professor',
        'education professional': 'professor',
        'főorvos': 'professor',
        'english trainer for ielts,toefl,pte,gre,sat exams.' : 'professor',
        'career coach' : 'professor',
        'hon dean' : 'professor',
        'academic specialist' : 'professor',


        'business partner': 'partner',
        'commercial consultant': 'partner',

        'c levelexecutive': 'c level executive',
        'c-levelexecutive' : 'c level executive',
        'c-level executive' : 'c level executive',
        
        'chief executive officer': 'c level executive',

        'entrylevel': 'entry level',


        'lider de desarrollo': 'manager',
        'administrative' : 'manager',
        'market intelligence/research': 'manager',
        'operations': 'manager',


        'business development/sales': 'associate/analyst',

        'principal at oxford integrated pu science college': 'director',
        'professional trainer': 'director',

        'unpaid': 'intern',

        'exhibitiontv': 'exhibition',

        'proprietário(a)': 'homeowner',
        'proprietário a': 'homeowner',

        'this is a consume display requirement for home purpose':'customer',
        'end user': 'customer',
        'end-user' : 'customer',
        'commercial end user' : 'customer',
        'commercial end-user' : 'customer',


        'surgery professional': 'hospital',
        'tierarzt' : 'hospital',
        'pathologist' : 'hospital',
        'radiology professional' : 'hospital',

        'medical imaging specialist': 'manufacturer',
        'business development/sales': 'manufacturer',
        'sales': 'manufacturer',
        'carge': 'manufacturer',


        'research': 'software/solution provider',
        'software /solution provider':'software/solution provider',

        'engineering': 'system integrator',

        'technical' : 'technician',
        'técnico' : 'technician',
    }, inplace=True)

    df['customer_position'] = df['customer_position'].str.lower()
    df.loc[df['customer_position'].str.contains('teacher'), 'customer_position'] = 'professor'
    df.loc[df['customer_position'].str.contains('professor'), 'customer_position'] = 'professor'
    df.loc[df['customer_position'].str.contains('lecturer'), 'customer_position'] = 'professor'
    df.loc[df['customer_position'].str.contains('pgt'), 'customer_position'] = 'professor'
    df.loc[df['customer_position'].str.contains('faculty'), 'customer_position'] = 'professor'

    df.loc[df['customer_position'].str.contains('management'), 'customer_position'] = 'manager'

    df.loc[df['customer_position'].str.contains('director'), 'customer_position'] = 'director'

    df.loc[df['customer_position'].str.contains('association'), 'customer_position'] = 'associate/analyst'

    df.loc[df['customer_position'].str.contains('sales'), 'customer_position'] = 'manufacturer'
    df.loc[df['customer_position'].str.contains('manufacturer'), 'customer_position'] = 'manufacturer'

    df['customer_position'] = df['customer_position'].apply(process_etc)

pre_train['customer_position'] = pre_train['customer_position'].astype(str)
pre_test['customer_position'] = pre_test['customer_position'].astype(str)
# 학습 데이터프레임 처리
process_position(pre_train)

# 테스트 데이터프레임 처리
process_position(pre_test)

In [18]:
pre_train['customer_position'].unique()

array(['entry level', 'ceo founder', 'partner', 'manager',
       'vice president', 'associate/analyst', 'c level executive', nan,
       'director', 'intern', 'trainee', 'customer', 'consultant',
       'professor', 'founder', 'system integrator', 'installer',
       'homeowner', 'employee', 'bulgaria', 'decision-influencer',
       'technician', 'exhibition', 'hospital', 'government',
       'manufacturer', 'software/solution provider', 'distributor',
       'cargo', 'english trainer for ielts toefl pte gre sat exams'],
      dtype=object)

In [19]:
translator = Translator()

def remove_special_characters(text):
    return re.sub(r'\W+', ' ', text)

def is_english(sentence):
    if sentence is np.nan:
        return sentence
    
    non_alpha_chars = re.sub(r'\W+', '', sentence)
    print(non_alpha_chars)
    non_alpha_chars = re.search(r'[^a-zA-Z]', non_alpha_chars)
    print(non_alpha_chars)
    
    if non_alpha_chars:
        changed_chars = translator.translate(sentence,dest='en').text 
#         return True
    else:
        changed_chars = sentence
    
    return changed_chars

# 특수 기호가 포함된 문자열 예시
n = np.nan
text_with_special_characters_1 = 'systèmes de débit à réfrigérant variable (drv)'
text_with_special_characters_2 = '醫院電視'
text_with_special_characters_3 = 'refrigerator,built-in/cooking'

# 특수 기호를 제거한 문자열 출력
print(is_english(text_with_special_characters_1))

systèmesdedébitàréfrigérantvariabledrv
<re.Match object; span=(4, 5), match='è'>
variable refrigerant flow (drv) systems


## 전처리 함수

In [20]:
def en_dic(df,col):
    unique_value = df[col].unique()
    trans_dic = {}
    
    for sen in unique_value:
        if sen is np.nan:
            trans_dic[sen] = sen
            continue

        non_alpha_chars = re.sub(r'\W+', '', sen)
        non_alpha_chars = re.search(r'[^a-zA-Z0-9]', non_alpha_chars)

        if non_alpha_chars:
            changed_chars = translator.translate(sen, dest='en').text 
            trans_dic[sen] = changed_chars
        
    return trans_dic

def to_english(df,col):
    # en_dic에 있는 값이면 해당 행 값을 사전의 값으로 교체
    df[col] = df[col].apply(lambda x: dic[x] if x in dic else x)

#//india// 등 손실되는 데이터 존재. -> 전처리 방법 수정
def process_country(country):
    result = np.nan
    
    if pd.isna(country):
        return np.nan
    
    parts = [x for x in country.split('/') if x != '']
    
    if len(parts) >= 1:
        result = parts[-1]
    else:
        return np.nan
    
    if '@' in str(result):
        return np.nan
        
    return result


# 묶을 수 있는 직업은 처리. 세분화 되어있더라도 모델에게는 아예 다른 직업으로 인식될 것.
def preprocess_job(job):
    if pd.isna(job):
        return np.nan
    
    if 'direct' in job:
        return 'director'
    elif 'consult' in job:
        return 'consultant'
    elif 'owner' in job:
        return 'owner'
    elif 'market' in job:
        return 'marketor'
    elif 'account' in job:
        return 'account'
    elif 'manage' in job:
        return 'manager'
    elif 'admin' in job:
        return 'administrator'
    elif 'design' in job:
        return 'designer'
    elif 'engineer' in job:
        return 'engineer'
    elif 'manager' in job:
        return 'manager'
    elif 'sales' in job:
        return 'sales'
    elif 'research' in job:
        return 'research'
    elif 'support' in job:
        return 'support'
    elif 'consultant' in job:
        return 'consultant'
    elif 'finance' in job:
        return 'finance'
    elif 'sale' in job:
        return 'sales'
    elif 'educat' or 'teach' in job:
        return 'educator'
    elif 'healthcare'in job:
        return 'healthcare'
    elif 'human' or 'hr posting' in job:
        return 'hr'
    elif 'inform' in job:
        return 'it'
    elif 'military' in job:
        return 'military'
    elif 'comunica' in job:
        return 'comunicator'
    else:
        return job
    
def process_type_mapping(df):
  # 사이트 참고하여 딕셔너리 구성
  # Customer Sub Type로 대체되어 작성된 경우 customer type으로 변경
  customer_type_mapping =  {
    'endcustomer' : ['construction company', 'owner', 'homeowner', 'developer', 'medical/healthcare facility',
                      'goverment/public sector', 'corporate', 'education', 'retail', 'fitness', 'manager/director', 'administrator'],

    'channelpartner' : ['agent','distributor','reseller','nsp','national reseller','regional reseller', 'si', 'pro', 'var'],

    'specifier/influencer' : ['architect', 'consultant', 'contractor', 'technical/design firm', 'region builder', 'installer', 'ad&contents provider', 'applied rep'],

    'solutionecopartner' : ['cms/webos/isv', 'mount/metal fabrication','meeting solution', 'control/processor', 'external compute'],

    'other' : ['hvacengineer', 'engineer', 'technician', 'systemintegrator'] # 분류를 어떻게 해야할 지 모르는 train unique한 값들 >> 따로 빼는 게 모델 점수 더 높음
    }
  for new_type, keywords in customer_type_mapping.items():
      df.loc[df['customer_type'].str.contains('|'.join(keywords), case=False), 'customer_type'] = new_type

In [21]:
need_to_en = ['customer_job' , 'product_category' , 'product_subcategory' , 'product_modelname']
for n in need_to_en:
    dic = en_dic(pre_train,n)
    to_english(pre_train,n)
    
for n in need_to_en:
    dic = en_dic(pre_test,n)
    to_english(pre_test,n)
    
pre_train['customer_country'] = pre_train['customer_country'].apply(process_country)
pre_test['customer_country'] = pre_test['customer_country'].apply(process_country)
pre_train['customer_country'] = pre_train['customer_country'].replace("us", "unitedstates")
pre_test['customer_country'] = pre_test['customer_country'].replace("us", "unitedstates")

pre_train['customer_job'].fillna(0)
pre_train['customer_job'] = pre_train['customer_job'].apply(preprocess_job)
pre_test['customer_job'] = pre_test['customer_job'].apply(preprocess_job)
                                                                        
pre_train['customer_type'] = pre_train['customer_type'].astype(str)
pre_test['customer_type'] = pre_test['customer_type'].astype(str)

# Customertype
process_type_mapping(pre_train)
process_type_mapping(pre_test)
    
    
# 'customer_position', 'customer_type' 열에서 특정 문자열을 'other'로 대체
columns_to_replace_other = ['customer_position', 'customer_type']
for col in columns_to_replace_other:
    pre_train[col] = pre_train[col].replace(['other','other_', 'etc.', 'others'], 'other')
    pre_test[col] = pre_test[col].replace(['other','other_', 'etc.', 'others'], 'other')

## product_category
### 전처리

In [22]:
pre_train['product_category'].unique()

array(['multi split', 'single split', 'vrf', 'chiller', 'other', 'rac',
       'teto ou cassete inverter', 'software solution',
       'all lg vrf systems', 'ventilation',
       'a thermodynamic water heater', 'residential air conditioner',
       'outros', 'heating', 'multi v5 vrf', 'split tunggal',
       'multi inverter', 'Heating', 'standard signage',
       'high brightness signage', 'interactive signage',
       'video wall signage', 'led signage', 'hotel tv',
       'signage care solution', 'oled signage', 'idb', 'video wall',
       'one:quick series', 'commercial tv', 'lg one:quick',
       'special signage', 'accessories', 'commercial display',
       'interactive digital board', 'standard', 'uhd signage',
       'smart tv signage', 'lg one:quick series', 'pro:centric', 'ur640',
       'ur640s', 'webos', 'signage care solutions', 'aio | one quick',
       'high brightness', 'hospital tv', 'digital signage', 'monitor',
       'pc', 'laptop', 'projector', 'cloud device', 'medi

In [23]:
def preprocess_pro_category(category):
    if pd.isna(category):
        return np.nan

    category_mapping = {
        'signage': ['awzm', 'lg led bloc', 'gsca', 'gscd', 'lsca039'],
        'tv': ['awz', 'bwz', 'tv signage', 'digital signage or commercial tvs', 'htv', 'tv 60',
               'ctv','tv55', 'tv43pol', 'ur640', 'ur640s', '50uq801c0sb.bwz',
               'uh', 'vl', 'high inch 86 / 98 or 110', '43us660h', 'hoteleria us670h','tv 55'],
        'system ac': ['cac', 'sac', 'vrfcentralairconditioner', 'aire acondicionado residencial', 'systemac', 'onorseveralreversibleac', 'ar condicionado residencial'],
        'vrf': ['vrf', 'variablerefrigerantflow(drv)systems'],
        'video wall': ['wall', '49vl5g', 'vm', '55svh7f a'],
        'procentric': ['centric'],
        'monitor': ['pol', 'monitor', '28mq780', '55tc3d'],
        'multiv': ['multiv'],
        'display': ['display'],
        'audio': ['audio'],
        'aio': ['aio'],
        'led': ['led', 'gscd046', 'laec'],
        'chiller': ['chill'],
        'one-quick': ['one', 'quick'],
        'washing machine': ['washing'],
        'multi-split': ['multi', 'split'],
        'single-split': ['single', 'split'],
        'projector': ['bu50nst'],
        'board': ['board', 'tr3'],
        'airsolution': ['airsolution', 'acrumah'],
        'other': ['etc.','Other']
    }

    for key, values in category_mapping.items():
        if any(value in category for value in values):
            return key

    return category

pre_train['product_category'] = pre_train['product_category'].apply(preprocess_pro_category)
pre_test['product_category'] = pre_test['product_category'].apply(preprocess_pro_category)

print(len(pre_train['product_category'].unique()))


107


In [24]:
pre_train['product_category'].unique()

array(['multi-split', 'vrf', 'chiller', 'other', 'rac',
       'teto ou cassete inverter', 'software solution', 'ventilation',
       'a thermodynamic water heater', 'one-quick', 'outros', 'heating',
       'Heating', 'standard signage', 'high brightness signage',
       'interactive signage', 'video wall', 'led', 'hotel tv',
       'signage care solution', 'idb', 'commercial tv', 'special signage',
       'accessories', 'display', 'board', 'standard', 'tv', 'procentric',
       'webos', 'signage care solutions', 'aio', 'high brightness',
       'hospital tv', 'digital signage', 'monitor', 'pc', 'laptop',
       'projector', 'cloud device', 'washing machine', 'dryer', 'solar',
       'system ac', 'aircare', 'built in/cooking', 'ess', 'robot', 'ems',
       'audio', 'ultra stretch series', 'fhd series', 'robots', 'id',
       'lg customer care program', 'lainnya', 'control', 'pendingin',
       'Chiller cooler', 'ahu', 'lg home bliss air solution',
       'lg paradise air solution', 'te

In [29]:
product_category_dict = {
    # B2B Product Inquiry
    'commmercial display' : ['oled signage', 'led signage', 'video wall signage', 'interactive signage',
                             'high brightness signage','special signage', 'standard signage', 'hotel tv',
                             'hospital tv', 'accessories', 'software solution', 'signage care solution',
                             'webos', 'pro:centric', 'one:quick series'],

    'it products' : ['monitor', 'laptop', 'projector', 'cloud device', 'medical display'],

    'hvac/ess' : ['control', 'ventilation', 'vrf', 'multi split', 'single split', 'chiller', 'heating'],

    'commmercial laundry' : ['titan','giant'],

    'compressor&motor' : ['compressor', 'reciprocating compressor', 'rotary compressor', 'scroll compressor', 'motor'],

    'robot' : ['lg cloi carrybot', 'lg cloi servebot', 'lg cloi servebot', 'lg cloi guidebot'],

    'advanced materials' : ['antimicrobial', 'porcelain enamel', 'specialty glass'],

    # Integrated Solution Inquiry
    'integrated solutions' : ['monitor', 'signage', 'commercial tv', 'ess', 'monior monitor tv',
                              'pc', 'projector', 'robot', 'system ac', 'ems', 'rac', 'chiller', 'tv',
                              'refrigerator', 'washing machine', 'aircare', 'vacuum cleaner', 'styler',
                              'dryer', 'built in cooking', 'home beauty', 'water care', 'audio video']
    # === monitor, robot, chiller이 B2B Product Inquiry과 중복 ===
    }

In [30]:
def process_product_category(text):
  result = text

  if pd.isna(text):
    return 'other'

  for new_category, keywords in product_category_dict.items():
    if text == new_category or text in keywords:
      return new_category

  return 'other'

process_product_category("tv")

'integrated solutions'

In [31]:
# product_category 상위 범주
pre_train['product_category_0'] = pre_train['product_category'].apply(process_product_category)
pre_test['product_category_0'] = pre_test['product_category'].apply(process_product_category)

In [32]:
pre_train['product_category_0'].unique()

array(['other', 'hvac/ess', 'integrated solutions', 'commmercial display',
       'it products', 'robot', 'compressor&motor'], dtype=object)

In [ ]:
print(pre_train[pre_train['product_category_0']=='commmercial display']['product_category'].unique())

In [ ]:
pre_train['product_category'].unique()

In [ ]:
pre_train['product_category_0'].unique()

In [ ]:
pre_train['product_category_0'].value_counts()

## time_line

In [ ]:
def is_period(text):
  if pd.isna(text):
      return np.nan
  for keyword in ['month', 'year']:
    if keyword in text:
      return text
    else:
      continue
  return np.nan

In [ ]:
# 기간으로 표현할 수 있는 데이터만 뽑아냄
period_timeline_train = pre_train['expected_timeline'].apply(is_period).copy()
period_timeline_test = pre_test['expected_timeline'].apply(is_period).copy()

period_timeline_train

In [ ]:
def extract_numbers(text):
  # 'a', 'one', 'next'을 숫자 1로 대체
  text = re.sub(r'\ba\b', '1', text)
  text = re.sub(r'\bone\b', '1', text)
  text = re.sub(r'\bnext\b', '1', text)

  numbers = re.findall(r'\d+', text)

  cleaned_numbers = []
  for number in numbers:
    # 숫자 뒤에 'month'나 'year'가 있는지 확인하고 number list에 담아줌
    # year이면 12개월이므로 12를 곱해줌
    if re.search(r'\b{}\s*(month)s?\b'.format(number), text):
      cleaned_numbers.append(float(number))
    elif re.search(r'\b{}\s*(year)s?\b'.format(number), text):
      cleaned_numbers.append(float(number)*12)

  return text, cleaned_numbers

t, n = extract_numbers('9 months 2 years')
t, n

In [ ]:
def timeline_to_number(text):
  if pd.isna(text):
        return np.nan

  # 문자열에서 숫자 추출
  _, numbers = extract_numbers(text)
  if len(numbers) == 1:
    num = numbers[0]
    if 'less than' in text:
      return num / 2
    elif 'more than' in text: # 6개월 후로 바꿔줌
      return (6 + num)
    else:
      return num

  elif len(numbers) == 2: # 기간일 때
      num1 = numbers[0]
      num2 = numbers[1]
      return (num1 + num2) / 2

  else:
      return np.nan  # 처리할 수 없는 경우 np.nan 반환

timeline_to_number('9 months 2 years')

In [ ]:
period_timeline_train = period_timeline_train.apply(timeline_to_number)
period_timeline_test = period_timeline_test.apply(timeline_to_number)

period_mean_train = round(period_timeline_train.mean(), 3)
period_mean_test = round(period_timeline_test.mean(), 3)
print(period_mean_train, period_mean_test)

In [ ]:
# object_columns = object_columns + columns_categorized
numerical_features = list(set(pre_train.columns) - set(object_columns))
numerical_features

In [ ]:
pre_train['expected_timeline_num'] = period_timeline_train.fillna(period_mean_train)
pre_test['expected_timeline_num'] = period_timeline_test.fillna(period_mean_test)
numerical_features.append('expected_timeline_num')

In [ ]:
pre_train['expected_timeline_num'].unique()

In [ ]:
pre_test['expected_timeline_num'].unique()

## 스케일링

In [ ]:
features = ['com_reg_ver_win_rate','lead_desc_length','ver_win_ratio_per_bu','historical_existing_cnt']

print(pre_train[features].min())
print(pre_train[features].max())

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

pre_train[features] = scaler.fit_transform(pre_train[features]) # fit_transform
pre_test[features] = scaler.transform(pre_test[features]) # transform

In [ ]:
print(pre_train[features].min())
print(pre_train[features].max())

In [ ]:
pre_train[features].head()

In [ ]:
pre_train.iloc[[1,2]]

In [ ]:
pre_train.info()

In [ ]:
# pre_train[categorical_features] = pre_train[categorical_features].fillna('other')
# pre_test[categorical_features] = pre_test[categorical_features].fillna('other')

categorical_features.append('country')
categorical_features.append('city')
# categorical_features.remove('customer_country')

categorical_features.append('product_category_0')

In [ ]:
categorical_features

## 3. 모델 학습

### 모델 학습

In [ ]:
# Stratified K-Fold 교차 검증 설정
n_splits = 5  # 원하는 Fold 수
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 최적의 하이퍼파라미터 설정
best_params = {'depth': 10, 'iterations': 1500, 'learning_rate': 0.01}

# 교차 검증 수행 및 평가 지표 저장
macro_f1_scores = []
models = []
best_f1_score = 0

resetIndex_train = pre_train.reset_index(drop=True)

train_x = resetIndex_train.drop(columns=['is_converted'])
train_y = resetIndex_train['is_converted'].astype(int)
train_x = train_x.fillna(0)

class_weights = compute_class_weight('balanced', classes=[0, 1], y=train_y)

for train_index, valid_index in skf.split(train_x, train_y):
    X_train, X_valid = train_x.iloc[train_index], train_x.iloc[valid_index]
    y_train, y_valid = train_y[train_index], train_y[valid_index]

    # CatBoost 모델 생성 및 학습
    model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes == np.object)[0], eval_metric='F1', 
                               random_seed=42, verbose=500, **best_params, class_weights = {0:class_weights[0], 1:class_weights[1]})
    
    model.fit(X_train, y_train, eval_set=[(X_valid,y_valid)])

    # 검증 데이터에 대한 예측 및 평가 (Macro F1 Score)
    valid_predictions = model.predict(X_valid)
    
    print(valid_predictions)
    print(y_valid[:5])
    macro_f1 = f1_score(y_valid, valid_predictions, average='macro')
    macro_f1_scores.append(macro_f1)
    models.append(model)

    if macro_f1 > best_f1_score:
        best_f1_score = macro_f1
        best_classification_model = model
        y_val = y_valid
        pred_val = valid_predictions

# Macro F1 Score 출력
print("Macro F1 Scores:", macro_f1_scores)


In [ ]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[1, 0])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [ ]:
get_clf_eval(y_val, pred_val)

## 4. 제출하기

### 테스트 데이터 예측

customer_job , inquiry_type , product_category , product_subcategory , product_modelname

In [ ]:
pre_test['customer_job'].unique()

In [ ]:
for n in columns_categorized:
    pre_test[n] = pre_test[n].fillna(0)
    pre_test[n] = pre_test[n].astype('int').astype('object')

In [ ]:
# 예측에 필요한 데이터 분리
x_test = pre_test.drop(["is_converted"], axis=1)
x_test.head()

In [ ]:
train_x.head()

In [ ]:
x_test.info()

In [ ]:
test_pred = best_classification_model.predict(x_test.fillna(0))
print(test_pred)
sum(int(pred == 1) for pred in test_pred)

### 제출 파일 작성

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [ ]:
best_classification_model.get_feature_importance(prettified=True)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**